In [ ]:
import torch
from torch.nn import CrossEntropyLoss
import matplotlib.pyplot as plt
cmap = plt.get_cmap("viridis")

import config as u_config
from graph_models import FullGraphModel

from data_processing import DataProcessor
from manifold_funcs import manifold_test

device_type = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device_type)
dtype = torch.float32
batch_size = u_config.batch_size

In [ ]:
data_processor = DataProcessor(u_config)
model = FullGraphModel(data_processor, u_config).to(device)
criterion = CrossEntropyLoss()

checkpoint = torch.load(
    "models/m_2024-07-24 17:10_xv8naljn.pth",  # for points
    map_location=device_type,
)
model.load_state_dict(checkpoint["model"])
torch.set_grad_enabled(False)
model.eval()

In [ ]:
test_results, final_plots, accuracy, intermediate, labels = manifold_test(
    model, data_processor, criterion, device, u_config
)

In [ ]:
intermediate = intermediate.cpu().numpy()

test_results["colour"] = test_results["Image"].str.split("/").str.get(-2)

In [ ]:
from sklearn.manifold import TSNE

# Perform t-SNE to reduce to 2D for visualization
tsne = TSNE(n_components=3, random_state=42)
reduced_data = tsne.fit_transform(intermediate)
test_results["tsne_Component_1"] = reduced_data[:, 0]
test_results["tsne_Component_2"] = reduced_data[:, 1]
test_results["tsne_Component_3"] = reduced_data[:, 2]

In [ ]:
import umap

# Perform UMAP to reduce to 2D for visualization
reducer = umap.UMAP(n_components=3)
reduced_data = reducer.fit_transform(intermediate)

test_results["umap_Component_1"] = reduced_data[:, 0]
test_results["umap_Component_2"] = reduced_data[:, 1]
test_results["umap_Component_3"] = reduced_data[:, 2]

In [ ]:
from sklearn.decomposition import PCA

pca_2d = PCA(n_components=3)
pca_2d_result = pca_2d.fit_transform(intermediate)

test_results["pca_Component_1"] = pca_2d_result[:, 0]
test_results["pca_Component_2"] = pca_2d_result[:, 1]
test_results["pca_Component_3"] = pca_2d_result[:, 2]

# 2D

In [ ]:
%matplotlib inline
def plot_2D(test_results, algorithm="tsne"):
    plt.figure(figsize=(10, 8))

    # Extract the first two t-SNE components and the color labels
    tsne_1 = test_results[f"{algorithm}_Component_1"]
    tsne_2 = test_results[f"{algorithm}_Component_2"]
    colours = test_results["colour"]

    # Create a scatter plot
    scatter = plt.scatter(tsne_1, tsne_2, c=colours, cmap="viridis", alpha=0.7)

    # Add a legend
    legend = plt.legend(*scatter.legend_elements(), title="Classes")
    plt.gca().add_artist(legend)

    # Labels and title
    plt.xlabel(f"{algorithm} Component 1")
    plt.ylabel(f"{algorithm} Component 2")
    plt.title(f"{algorithm} Plot of Test Results")

    plt.grid(True)
    plt.show()

In [ ]:
plot_2D(test_results, algorithm="tsne")

In [ ]:
plot_2D(test_results, algorithm="umap")

In [ ]:
plot_2D(test_results, algorithm="pca")

In [ ]:
from manifold_plots import plot_manifold_3d

def plot_manifold_3d(test_results, geom="shape", algorithm="tsne"):
    if geom == "shape":
        shape_colors = {"circle": "blue", "star": "red"}
    elif geom == "colour":
        shape_colors = {"blue": "blue", "yellow": "yellow"}

    # Create the 3D scatter plot
    fig = go.Figure()

    # Add scatter3d trace for color coding
    for shape in test_results[geom].unique():
        df_shape = test_results[test_results[geom] == shape]
        fig.add_trace(
            go.Scatter3d(
                x=df_shape[f"{algorithm}_Component_1"],
                y=df_shape[f"{algorithm}_Component_2"],
                z=df_shape[f"{algorithm}_Component_3"],
                mode="markers",
                marker=dict(
                    size=2,
                    color=shape_colors[shape],
                    opacity=0.8,
                    line=dict(width=0),  # Remove the white border of the points
                ),
                name=shape,
            )
        )

    # Update layout to remove the background and set other properties
    fig.update_layout(
        scene=dict(
            xaxis=dict(backgroundcolor="rgba(0,0,0,0)", showgrid=False, zeroline=False),
            yaxis=dict(backgroundcolor="rgba(0,0,0,0)", showgrid=False, zeroline=False),
            zaxis=dict(backgroundcolor="rgba(0,0,0,0)", showgrid=False, zeroline=False),
        ),
        title=f"{algorithm.upper()} of Decision Making Neuron Activations",
        showlegend=True,
    )

    # Show the plot
    fig.show()

In [ ]:
import plotly.graph_objects as go

plot_manifold_3d(test_results, geom="colour", algorithm="tsne")

In [ ]:
plot_manifold_3d(test_results, geom="colour", algorithm="umap")

In [ ]:
plot_manifold_3d(test_results, geom="colour", algorithm="pca")